# 실습 | 네이버 영화 리뷰

## 데이터 불러오기

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

In [ ]:
data = pd.read_table("ratings_train.txt")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [ ]:
data.loc[data.document.isnull()]
data = data.dropna()

In [ ]:
data.loc[data.document.isnull()]

,id,document,label


## 전체 데이터 나이브베이즈 실행

In [ ]:
cv = CountVectorizer()
x_train = cv.fit_transform(data['document'])
y_train = data['label']

In [ ]:
nb = MultinomialNB()
nb.fit(x_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [ ]:
txt = pd.Series('행복한 하루를 보내고 있어요')
x_test = cv.transform(txt)
nb.predict(x_test)

array([1])

In [ ]:
txt = pd.Series('오늘 하루는 짜증이 난다.')
x_test = cv.transform(txt)
nb.predict(x_test)

array([0])

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_train, nb.predict(x_train)))

              precision    recall  f1-score   support

           0       0.95      0.94      0.94     75170
           1       0.94      0.95      0.94     74825

    accuracy                           0.94    149995
   macro avg       0.94      0.94      0.94    149995
weighted avg       0.94      0.94      0.94    149995



## 훈련 데이터 / 테스트 데이터 나누기

In [ ]:
X = data['document']
Y = data['label']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)

In [ ]:
from collections import Counter
Counter(Y_train)

Counter({0: 59958, 1: 60038})

In [ ]:
Counter(Y_test)

Counter({0: 15212, 1: 14787})

## 나이브베이즈 실행

In [ ]:
cv = CountVectorizer()
X_train = cv.fit_transform(X_train)

In [ ]:
nb = MultinomialNB()
nb.fit(X_train, Y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [ ]:
X_test = cv.transform(X_test)

In [ ]:
print(classification_report(Y_test, nb.predict(X_test)))

              precision    recall  f1-score   support

           0       0.87      0.78      0.82     15212
           1       0.80      0.87      0.83     14787

    accuracy                           0.83     29999
   macro avg       0.83      0.83      0.83     29999
weighted avg       0.83      0.83      0.83     29999



## 형태소 분석 

In [ ]:
from nltk.tokenize import word_tokenize
import nltk
nltk.download("all")

In [ ]:
!pip install konlpy

In [ ]:
from konlpy.tag import Okt
pos_tagger = Okt()

In [ ]:
X = [pos_tagger.morphs(i) for i in data['document']]
X[0]

['아', '더빙', '..', '진짜', '짜증나네요', '목소리']

In [ ]:
contents_vectorize = []

for i in X:
  txt = ''
  for w in i:
    txt = txt + " " + w
  contents_vectorize.append(txt)

contents_vectorize

[' 아 더빙 .. 진짜 짜증나네요 목소리',
 ' 흠 ... 포스터 보고 초딩 영화 줄 .... 오버 연기 조차 가볍지 않구나',
 ' 너 무재 밓었 다그 래서 보는것을 추천 한 다',
 ' 교도소 이야기 구먼 .. 솔직히 재미 는 없다 .. 평점 조정',
 ' 사이 몬페 그 의 익살스런 연기 가 돋보였던 영화 ! 스파이더맨 에서 늙어 보이기만 했던 커스틴 던스트 가 너무나도 이뻐 보였다',
 ' 막 걸음 마 뗀 3 세 부터 초등학교 1 학년 생인 8 살용 영화 . ㅋㅋㅋ ... 별 반개 도 아까 움 .',
 ' 원작 의 긴장감 을 제대로 살려내지못 했다 .',
 ' 별 반개 도 아깝다 욕 나온다 이응경 길용우 연 기 생활 이 몇 년 인지 .. 정말 발 로 해도 그것 보단 낫겟다 납치 . 감금 만 반복 반복 .. 이 드라마 는 가족 도 없다 연기 못 하는 사람 만 모 엿 네',
 ' 액션 이 없는데도 재미 있는 몇 안되는 영화',
 ' 왜케 평점 이 낮은건데 ? 꽤 볼 만 한 데 .. 헐리우드 식 화려함에만 너무 길들여져 있나 ?',
 ' 걍 인피니트 가 짱 이다 . 진짜 짱 이다 ♥',
 ' 볼때 마다 눈물나서 죽겠다 90년 대의 향수 자극 !! 허진호 는 감성 절제 멜로 의 달인 이다 ~',
 ' 울면 서 손 들 고 횡단보도 건널 때 뛰 쳐나 올 뻔 이범수 연기 드럽게 못 해',
 ' 담백하고 깔끔해서 좋다 . 신 문 기 사 로만 보다 보면 자꾸 잊어버린다 . 그 들 도 사람 이었다는 것 을 .',
 ' 취향 은 존중 한 다지 만 진짜 내생 에 극장 에서 본 영화 중 가장 노잼 노 감동 임 스토리 도 어거지 고 감동 도 어거지',
 ' ㄱ 냥 매번 긴장 되고 재밋음 ㅠㅠ',
 ' 참 사람 들 웃긴게 바스코 가 이기 면 락스 코 라고 까고 바비 가 이기 면 아이돌 이라고 깐다 . 그냥 까고싶어서 안달 난 것 처럼 보인다',
 ' 굿바이 레닌 표절 인 것 은 이해 하는데 왜 뒤 로 갈수록 재미 없어지냐',
 ' 이건 정말 깨알 캐스팅 과 질퍽 하 지 않은 산

In [ ]:
Y = data['label']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(contents_vectorize, Y, test_size = 0.2)

In [ ]:
from collections import Counter
Counter(Y_train)

Counter({0: 60035, 1: 59961})

In [ ]:
Counter(Y_test)

Counter({0: 15135, 1: 14864})

In [ ]:
cv = CountVectorizer()
X_train = cv.fit_transform(X_train)
X_train

<119996x88186 sparse matrix of type '<class 'numpy.int64'>'
	with 987271 stored elements in Compressed Sparse Row format>

In [ ]:
nb = MultinomialNB()
nb.fit(X_train, Y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [ ]:
X_test = cv.transform(X_test)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(Y_test, nb.predict(X_test)))

              precision    recall  f1-score   support

           0       0.84      0.84      0.84     15135
           1       0.84      0.84      0.84     14864

    accuracy                           0.84     29999
   macro avg       0.84      0.84      0.84     29999
weighted avg       0.84      0.84      0.84     29999



## 전처리 작업을 통한 정확도 향상

In [ ]:
import re

In [ ]:
re.sub("[^가-힣]", " ", data['document'][0])

'아 더빙   진짜 짜증나네요 목소리'

In [ ]:
data['document'] = data['document'].str.replace("[^가-힣]", " ")

In [ ]:
data['document'][0:5]

0                                  아 더빙   진짜 짜증나네요 목소리
1                    흠   포스터보고 초딩영화줄    오버연기조차 가볍지 않구나
2                                    너무재밓었다그래서보는것을추천한다
3                        교도소 이야기구먼   솔직히 재미는 없다  평점 조정
4    사이몬페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어보이기만 했던 커스틴 ...
Name: document, dtype: object

## 불용어 처리

In [ ]:
stopword = []

with open("stopwords_kr.txt") as file:
  for word in file:
    stopword.append(word.strip())

In [ ]:
stopword

['아',
 '휴',
 '아이구',
 '아이쿠',
 '아이고',
 '어',
 '나',
 '우리',
 '저희',
 '따라',
 '의해',
 '을',
 '를',
 '에',
 '의',
 '가',
 '으로',
 '로',
 '에게',
 '뿐이다',
 '의거하여',
 '근거하여',
 '입각하여',
 '기준으로',
 '예하면',
 '예를 들면',
 '예를 들자면',
 '저',
 '소인',
 '소생',
 '저희',
 '지말고',
 '하지마',
 '하지마라',
 '다른',
 '물론',
 '또한',
 '그리고',
 '비길수 없다',
 '해서는 안된다',
 '뿐만 아니라',
 '만이 아니다',
 '만은 아니다',
 '막론하고',
 '관계없이',
 '그치지 않다',
 '그러나',
 '그런데',
 '하지만',
 '든간에',
 '논하지 않다',
 '따지지 않다',
 '설사',
 '비록',
 '더라도',
 '아니면',
 '만 못하다',
 '하는 편이 낫다',
 '불문하고',
 '향하여',
 '향해서',
 '향하다',
 '쪽으로',
 '틈타',
 '이용하여',
 '타다',
 '오르다',
 '제외하고',
 '이 외에',
 '이 밖에',
 '하여야',
 '비로소',
 '한다면 몰라도',
 '외에도',
 '이곳',
 '여기',
 '부터',
 '기점으로',
 '따라서',
 '할 생각이다',
 '하려고하다',
 '이리하여',
 '그리하여',
 '그렇게 함으로써',
 '하지만',
 '일때',
 '할때',
 '앞에서',
 '중에서',
 '보는데서',
 '으로써',
 '로써',
 '까지',
 '해야한다',
 '일것이다',
 '반드시',
 '할줄알다',
 '할수있다',
 '할수있어',
 '임에 틀림없다',
 '한다면',
 '등',
 '등등',
 '제',
 '겨우',
 '단지',
 '다만',
 '할뿐',
 '딩동',
 '댕그',
 '대해서',
 '대하여',
 '대하면',
 '훨씬',
 '얼마나',
 '얼마만큼',
 '얼마큼',
 '남짓',
 '여',
 '얼마간',
 '약간',
 '다소',
 '좀',
 '조

In [ ]:
from konlpy.tag import Okt
pos_tagger = Okt()

In [ ]:
def text_tokenizing(doc):
  return [word for word in pos_tagger.morphs(doc) if word not in stopword and len(word) > 1]

In [ ]:
contents_token = [text_tokenizing(i) for i in data['document']]

In [ ]:
contents_token[0]

['더빙', '진짜', '짜증나네요', '목소리']

In [ ]:
contents = [" ".join(i) for i in contents_token]
contents[0:2]

['더빙 진짜 짜증나네요 목소리', '포스터 보고 초딩 영화 오버 연기 가볍지 않구나']

In [ ]:
X = contents
Y = data['label']

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)

In [ ]:
Counter(Y_train)

Counter({0: 60092, 1: 59904})

In [ ]:
Counter(Y_test)

Counter({0: 15078, 1: 14921})

In [ ]:
cv = CountVectorizer()
X_train = cv.fit_transform(X_train)
nb = MultinomialNB()
nb.fit(X_train, Y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [ ]:
X_test = cv.transform(X_test)
print(classification_report(Y_test, nb.predict(X_test)))

              precision    recall  f1-score   support

           0       0.82      0.85      0.83     15078
           1       0.84      0.82      0.83     14921

    accuracy                           0.83     29999
   macro avg       0.83      0.83      0.83     29999
weighted avg       0.83      0.83      0.83     29999

